In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.densenet import DenseNet201

In [ ]:
path_dataset = 'FaceMaskDataset\Train'
os.listdir(path_dataset)

In [ ]:
print('total WithMask images  :', 
      len(os.listdir(path_dataset + '/WithMask')))
print('total WithoutMask images :', 
      len(os.listdir(path_dataset + '/WithoutMask')))

**Image Generator**

In [ ]:
dataset_dir = os.path.join(path_dataset)

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    zoom_range = 0.2,
    shear_range = 0.2,
    fill_mode = 'nearest',
    validation_split = 0.2
)

valid_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size = (50, 50),
    batch_size = 128,
    class_mode = 'binary',
    color_mode = 'rgb',
    subset = 'training'
)

valid_generator = valid_datagen.flow_from_directory(
    dataset_dir,
    target_size = (50, 50),
    batch_size = 128,
    class_mode = 'binary',
    color_mode = 'rgb',
    subset = 'validation'
)

**Callbacks**

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('val_accuracy') > 0.95):
            print("\nAccuracy Validasi telah mencapai > 95%!")
            self.model.stop_training = True

callbacks = myCallback()

es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)

**Modeling**

In [ ]:
model = tf.keras.models.Sequential([
    DenseNet201(weights='imagenet', include_top=False, input_shape=(50,50,3)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid') 
])

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

model.layers[0].trainable = False

model.summary()

In [ ]:
hist = model.fit(train_generator,
                 epochs = 10,
                 validation_data = valid_generator,
                 callbacks = [es],
                 verbose = 1)

**Plot Loss and Accuracy**

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))
ax[0].plot(hist.history['loss'], label='Data Training')
ax[0].plot(hist.history['val_loss'], label='Data Testing')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')
ax[0].legend()

ax[1].plot(hist.history['accuracy'], label='Data Training')
ax[1].plot(hist.history['val_accuracy'], label='Data Testing')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Accuracy')
ax[1].legend()

plt.tight_layout()
plt.show()

**Save Model HDF5 Format**

In [ ]:
model.save('mask_model/mask_model.h5')

**Save Model to TF-Lite**

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with tf.io.gfile.GFile('mask_model/mask_model.tflite', 'wb') as f:
  f.write(tflite_model)